# **02.3 Data Cleaning - Merging Datasets**

In [ ]:
!pip install rapidfuzz

In [ ]:
import pandas as pd
from rapidfuzz import process, fuzz

In [28]:
# Define the file paths
path1 = '/lakehouse/default/Files/data/processed/processed_vehicle_energy_data.csv'
path2 = '/lakehouse/default/Files/data/processed/processed_vehicle_dimension_data.csv'

# Load the datasets into pandas dataframes
df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)

StatementMeta(, 44495aea-7991-43f5-a5ff-e5ee1525fab5, 30, Finished, Available)

In [29]:
df1.columns = [column.upper() for column in df1.columns]
df1.head()

StatementMeta(, 44495aea-7991-43f5-a5ff-e5ee1525fab5, 31, Finished, Available)

,MAKE,MODEL,YEAR,CITYE_KWH/100MI,HIGHWAYE_KWH/100MI,COMBE_KWH/100MI
0,Alfa Romeo,Spider Veloce 2000,1985,177.368421,134.800000,160.476190
1,Ferrari,Testarossa,1985,374.444444,240.714286,306.363636
2,Dodge,Charger,1985,146.521739,102.121212,124.814815
3,Dodge,B150/B250 Wagon 2WD,1985,337.000000,280.833333,306.363636
4,Subaru,Legacy AWD Turbo,1993,198.235294,146.521739,177.368421


In [30]:
df2.rename(columns={'Adjusted_MYR': 'YEAR'}, inplace=True)
df2.head()

StatementMeta(, 44495aea-7991-43f5-a5ff-e5ee1525fab5, 32, Finished, Available)

,MAKE,MODEL,MYR,OL,OW,OH,WB,CW,A1,B1,C1,D1,E1,F1,G1,TWF,TWR,WDIST,FILE_YEAR,YEAR
0,ACURA,ILX 4DR SEDAN/HYBRID,12.0,454.0,180.0,141.0,267.0,1356.0,121.0,51.0,30.0,79.0,114.0,96.0,91.0,150.0,152.0,60/40,2013,2012
1,ACURA,MDX 4DR SUV AWD/TECH/ELITE,10.0,485.0,199.0,173.0,275.0,2076.0,117.0,183.0,42.0,85.0,126.0,100.0,110.0,172.0,172.0,56/44,2013,2010
2,ACURA,TL 4 DR SEDAN FWD/TECH,12.0,493.0,188.0,145.0,278.0,1695.0,140.0,65.0,32.0,81.0,115.0,102.0,113.0,161.0,162.0,61/39,2013,2012
3,ACURA,TL 4 DR SEDAN SH-AWD/SH-AWD TECH/SH-AWD ELITE,12.0,493.0,188.0,145.0,278.0,1807.0,140.0,65.0,32.0,81.0,115.0,102.0,113.0,161.0,162.0,59/41,2013,2012
4,ACURA,TSX 4DR SEDAN FWD TECH PACKAGE/PREMIUM,9.0,473.0,184.0,144.0,271.0,1545.0,121.0,44.0,35.0,79.0,115.0,97.0,104.0,158.0,158.0,60/40,2013,2009


In [31]:
for df in [df1, df2]:
    print(df.columns)
    # Remove special characters and standardize the case for MAKE
    df['MAKE'] = df['MAKE'].str.upper().str.strip().str.replace('[^a-zA-Z0-9 ]', '', regex=True)
    
    # Remove special characters, "2DR", "4DR", and standardize the case for MODEL
    df['MODEL'] = df['MODEL'].str.upper().str.strip().str.replace('[^a-zA-Z0-9 ]', '', regex=True).str.replace('2DR', '', regex=False).str.replace('4DR', '', regex=False)
    
    # Ensure YEAR is an integer
    df['YEAR'] = df['YEAR'].astype(int)



StatementMeta(, 44495aea-7991-43f5-a5ff-e5ee1525fab5, 33, Finished, Available)

Index(['MAKE', 'MODEL', 'YEAR', 'CITYE_KWH/100MI', 'HIGHWAYE_KWH/100MI',
       'COMBE_KWH/100MI'],
      dtype='object')
Index(['MAKE', 'MODEL', 'MYR', 'OL', 'OW', 'OH', 'WB', 'CW', 'A1', 'B1', 'C1',
       'D1', 'E1', 'F1', 'G1', 'TWF', 'TWR', 'WDIST', 'FILE_YEAR', 'YEAR'],
      dtype='object')


## Overview
The cell below explains the strategy for matching two datasets using a combination of string manipulation and similarity scoring. The approach involves creating a standardized, alphabetically ordered key from vehicle make, model, and year, then utilizing RapidFuzz to find matches based on string similarity.

## Strategy

### Creating a Combined Key
- **Concatenate** vehicle `MAKE`, `MODEL`, and `YEAR` into a single string.
- **Alphabetize** the characters of the combined string to create a uniform key across datasets.

### Matching Process
- **Apply** the combined key creation to both datasets.
- **Set** a similarity threshold (e.g., 80%) to identify matches.
- **Use** RapidFuzz to compare the combined keys for similarity scoring.
- **Collect** matches that meet or exceed the threshold.

### Merging Data
- **Sort** and **deduplicate** matches to keep the highest similarity for each index.
- **Merge** the datasets based on these matches.

## Considerations
- The method is **not perfect** but efficient for combining datasets with slight variations in naming.
- It was **sufficient** for progressing in the hackathon, highlighting a balance between accuracy and practicality.


In [37]:
# Function to create a standardized, alphabetically ordered combined key
def create_combined_key(row):
    # Concatenate MAKE, MODEL, and YEAR into a single string without spaces
    combined_string = f"{row['MAKE']}{row['MODEL']}{str(row['YEAR'])}"
    # Remove spaces and sort the characters in alphabetical order
    sorted_characters = sorted(combined_string.replace(" ", ""))
    # Join the sorted characters back into a single string
    combined_key = ''.join(sorted_characters)
    return combined_key


# Apply the function to both dataframes
for df in [df1, df2]:
    df['Combined_Key'] = df.apply(create_combined_key, axis=1)

# Set a similarity threshold
similarity_threshold = 80

# Collect match details in a list first
match_details = []

for index1, row1 in df1.iterrows():
    best_match = process.extractOne(row1['Combined_Key'], df2['Combined_Key'].tolist(), scorer=fuzz.ratio)
    if best_match and best_match[1] >= similarity_threshold:
        match_index2 = df2[df2['Combined_Key'] == best_match[0]].index.to_list()[0]
        match_details.append({'DF1_Index': index1, 'DF2_Index': match_index2, 'Similarity': best_match[1]})

# Convert the list of match details to a DataFrame
matches_df = pd.DataFrame(match_details)

# Assuming multiple matches can occur, we keep only the highest similarity match for each DF1_Index
matches_df = matches_df.sort_values(by=['DF1_Index', 'Similarity'], ascending=[True, False]).drop_duplicates(subset=['DF1_Index'])

# Merge based on the match details
# First, add a column to df1 that maps to df2's index
df1['DF2_Index'] = df1.index.map(matches_df.set_index('DF1_Index')['DF2_Index'])

# Then merge df1 and df2 based on this new column and df2's index
merged_df = pd.merge(df1, df2, left_on='DF2_Index', right_index=True, suffixes=('_df1', '_df2'))

# Now, merged_df contains merged information from df1 and df2 based on the highest similarity matches

StatementMeta(, 44495aea-7991-43f5-a5ff-e5ee1525fab5, 39, Finished, Available)

In [39]:
# merged_df.shape[0]
merged_df.sample(n=5)

StatementMeta(, 44495aea-7991-43f5-a5ff-e5ee1525fab5, 41, Finished, Available)

,MAKE_df1,MODEL_df1,YEAR_df1,CITYE_KWH/100MI,HIGHWAYE_KWH/100MI,COMBE_KWH/100MI,Combined_Key_df1,DF2_Index,MAKE_df2,MODEL_df2,...,D1,E1,F1,G1,TWF,TWR,WDIST,FILE_YEAR,YEAR_df2,Combined_Key_df2
20951,CHEVROLET,CORVETTE,2011,224.666667,140.416667,187.222222,0112CCEEEEHLOORRTTTVV,11845.0,CHEVROLET,CORVETTE COUPE,...,67.0,110.0,105.0,106.0,146.0,149.0,52/48,1995,1992,1299CCCEEEEEHLOOOPRRTTTUVV
23906,MERCEDESBENZ,C350 4MATIC COUPE,2012,177.368421,124.814815,153.181818,00122345ABCCCCDEEEEEIMMNOPRSTUZ,456.0,MERCEDESBENZ,CCLASS C350 4MATIC COUPE AWD,...,81.0,101.0,78.0,106.0,151.0,151.0,53/47,2013,2012,00122345AAABCCCCCCDDEEEEEILMMNOPRSSSTUWZ
24824,NISSAN,MAXIMA,2014,177.368421,129.615385,153.181818,0124AAAIIMMNNSSX,4754.0,NISSAN,MAXIMA SEDAN,...,85.0,113.0,102.0,112.0,158.0,158.0,61/39,2022,2016,0126AAAADEIIMMNNNSSSX
25997,BMW,X6 XDRIVE50I,2015,224.666667,153.181818,198.235294,0012556BDEIIMRVWXX,2936.0,BMW,X6 XDRIVE 50I SUV,...,93.0,115.0,90.0,108.0,164.0,170.0,52/48,2015,2015,0012556BDEIIMRSUVVWXX
33932,VOLKSWAGEN,ATLAS CROSS SPORT,2021,160.476190,140.416667,153.181818,0122AAACEGKLLNOOOPRRSSSSSTTVW,5645.0,VOLKSWAGEN,PASSAT 20L TSI SEDAN,...,79.0,113.0,91.0,114.0,156.0,153.0,60/40,2018,2011,001122AAAADEEGIKLLNNOPSSSSSTTVW


In [41]:
# Define the save path
save_path = '/lakehouse/default/Files/data/processed/merged_vehicle_data.csv'

# Save merged_df to CSV
merged_df.to_csv(save_path, index=False)


StatementMeta(, 44495aea-7991-43f5-a5ff-e5ee1525fab5, 43, Finished, Available)